# Token Counter Notebook

This notebook allows you to input text, retrieve outputs, and calculate token counts using OpenAI, Gemini, Anthropic, and Grok APIs.

## 1. Install and Import Required Libraries

Install necessary libraries such as `openai`, `anthropic`, and others. Import required modules like `os` and `json`.

In [1]:
# Install required libraries (uncomment if running for the first time)
# !pip install openai anthropic google-generativeai requests

import os
import json
import openai
import anthropic
from google import genai
import requests

In [2]:
# Make output cells take full width
from IPython.core.display import HTML
HTML("<style>.jp-OutputArea-output { max-width: 100vw !important; }</style>")

## 2. Set Up API Keys

Set up environment variables or directly define API keys for OpenAI, Gemini, Anthropic, and Grok.

In [3]:
# Set your API keys here or load them from environment variables

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "your-anthropic-api-key")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your-gemini-api-key")
GROK_API_KEY = os.getenv("GROK_API_KEY", "your-grok-api-key")

# Set API keys for respective libraries if possible
openai.api_key = OPENAI_API_KEY

In [4]:
# Enter your user prompt here
prompt = 'hello'

# Enter your system prompt here
system_prompt = ''

# Enter the number of trials to run
num_trials = 3

In [5]:
def process_with_openai_chat(prompt, system_prompt, model="gpt-4o"):
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        output = response.choices[0].message.content
        input_tokens = getattr(response.usage, 'prompt_tokens', None)
        output_tokens = getattr(response.usage, 'completion_tokens', None)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"OpenAI ChatCompletions error: {str(e)}", None, None

In [6]:
def process_with_gemini(prompt, system_prompt, model="gemini-2.0-flash"):
    try:
        from google import genai
        from google.genai import types
        client = genai.Client(api_key=GEMINI_API_KEY)
        response = client.models.generate_content(
            model=model,
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=prompt
        )
        output = getattr(response, 'text', str(response))
        usage = getattr(response, 'usage_metadata', None)
        input_tokens = usage.prompt_token_count if usage and hasattr(usage, 'prompt_token_count') else None
        output_tokens = usage.candidates_token_count if usage and hasattr(usage, 'candidates_token_count') else None
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Gemini error: {str(e)}", None, None

In [7]:
def process_with_anthropic(prompt, system_prompt, model="claude-3-7-sonnet-20250219"):
    try:
        import anthropic
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        message = client.messages.create(
            model=model,
            max_tokens=1024,
            system=system_prompt,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        output = message.content[0].text if hasattr(message.content[0], 'text') else str(message.content[0])
        input_tokens = getattr(message.usage, 'input_tokens', None)
        output_tokens = getattr(message.usage, 'output_tokens', None)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Anthropic error: {str(e)}", None, None

In [8]:
def process_with_grok(prompt, system_prompt, model="grok-3-beta"):
    try:
        from openai import OpenAI
        client = OpenAI(
            api_key=GROK_API_KEY,
            base_url="https://api.x.ai/v1",
        )
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        completion = client.chat.completions.create(
            model=model,
            messages=messages
        )
        output = completion.choices[0].message.content
        input_tokens = getattr(completion.usage, 'prompt_tokens', None)
        output_tokens = getattr(completion.usage, 'completion_tokens', None)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Grok error: {str(e)}", None, None

In [9]:
all_rows = []
for run in range(1, num_trials + 1):
    results = {}
    try:
        output, in_tok, out_tok = process_with_openai_chat(prompt, system_prompt)
        results['OpenAI'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok, 'model': 'gpt-4o'}
    except Exception as e:
        results['OpenAI'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None, 'model': 'gpt-4o'}
    try:
        output, in_tok, out_tok = process_with_gemini(prompt, system_prompt)
        results['Gemini'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok, 'model': 'gemini-2.0-flash'}
    except Exception as e:
        results['Gemini'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None, 'model': 'gemini-2.0-flash'}
    try:
        output, in_tok, out_tok = process_with_anthropic(prompt, system_prompt)
        results['Anthropic'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok, 'model': 'claude-3-7-sonnet-20250219'}
    except Exception as e:
        results['Anthropic'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None, 'model': 'claude-3-7-sonnet-20250219'}
    try:
        output, in_tok, out_tok = process_with_grok(prompt, system_prompt)
        results['Grok'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok, 'model': 'grok-3-beta'}
    except Exception as e:
        results['Grok'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None, 'model': 'grok-3-beta'}
    for vendor, res in results.items():
        all_rows.append({
            'Run Number': run,
            'Vendor': vendor,
            'Model': res['model'],
            'User Prompt': prompt,
            'System Prompt': system_prompt,
            'Output': res['output'],
            'Input Tokens': res['input_tokens'],
            'Output Tokens': res['output_tokens']
        })
import pandas as pd
df = pd.DataFrame(all_rows)
display(df)
# Output all results to a CSV file
csv_path = 'api_results.csv'
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f'Results output to {csv_path}')

,Run Number,Vendor,Model,User Prompt,System Prompt,Output,Input Tokens,Output Tokens
0,1,OpenAI,gpt-4o,hello,,Hello! How can I assist you today?,12,10
1,1,Gemini,gemini-2.0-flash,hello,,Hello there! How can I help you today?\n,1,11
2,1,Anthropic,claude-3-7-sonnet-20250219,hello,,Hello! How can I assist you today? I'm here to...,8,36
3,1,Grok,grok-3-beta,hello,,Hey there! How can I help you today? 😊,7,13
4,2,OpenAI,gpt-4o,hello,,Hello! How can I assist you today?,12,10
5,2,Gemini,gemini-2.0-flash,hello,,Hello there! How can I help you today?\n,1,11
6,2,Anthropic,claude-3-7-sonnet-20250219,hello,,Hello! How can I assist you today? I'm here to...,8,40
7,2,Grok,grok-3-beta,hello,,Hey there! How can I help you today?,7,11
8,3,OpenAI,gpt-4o,hello,,Hello! How can I assist you today?,12,10
9,3,Gemini,gemini-2.0-flash,hello,,Hello! How can I help you today?\n,1,10


Results output to api_results.csv
